# Task 4

## 1 
Factorize n = 275621053. You can assume that n = pq, where p−q is relatively small. Show
your calculation steps.


Since n-q is relatively small, these numbers must be close to each other. That means that they must also be close to the square root of n. I will start at sqrt(n) and go upwards/downwards on p/q (based on the product compared to n), and find two numbers that multiply to n

In [3]:
import math

n = 275621053
p, q = int(math.sqrt(n)), int(math.sqrt(n))
while p * q != n:
  if p * q < n:
    p += 1
  else:
    q -= 1
print("n =", n)
print("p =", p, "q =", q)
print("p * q =", p * q)

n = 275621053
p = 17021 q = 16193
p * q = 275621053


## Task 2

### a
Alice wants to set up her RSA encryption with private key (n, d) with n = pq, using two
primes p and q, and private key d = 3. She chooses p = 1283, but wonders which of the
following choices for q she should use (NB! They are all prime numbers):
$$ 1307, 1879, 2003, 2027 $$

Explain why she should use q = 2027 for the system to work and to be most secure. For
the weak choices of q, name an effective attack to factorize n (of course, these numbers
are far too small to be secure, so consider the security in relative terms.) <hr>


Since p = 1283, we want a number that is not so close to this. This means that 1307 is not an option. We then look at the other options for q. Both  (p-1) and (q-1) should not have small factors (other than 2 obviously). This is so that we make it more secure against Pollard's p-1 algorithm. We look at the factors for q-1:

In [9]:
def print_factors(x):
   print("The factors of",x,"are:")
   for i in range(2, int(x/2 + 1)):
       if x % i == 0:
           print(i)

numbers = [1879, 2003, 2027]
for number in numbers:
  print_factors(number-1)

The factors of 1878 are:
2
3
6
313
626
939
The factors of 2002 are:
2
7
11
13
14
22
26
77
91
143
154
182
286
1001
The factors of 2026 are:
2
1013


Here we see that 2026 (2027 - 1) is 2*1013, which is a prime multiplied with 2. This makes 2027 a good choice for q, since it's also not so similar to q (1283)

### b
Find the corresponding public key e using the extended Euclidean algorithm. Write a
program to do the calculation.<hr>
Calculate the multiplicative inverse d to e
modulo (𝑝 − 1) (𝑞 − 1) , 𝑠𝑜
𝑑𝑒 ≡ 1 𝑚𝑜𝑑 (𝑝 − 1)(𝑞 − 1)


In [24]:
def extended_euclidean(a, b):
    if a == 0:
        return b, 0, 1
    else:
        gcd, x1, y1 = extended_euclidean(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y
     

In [25]:
p = 1287
q = 2027
d = 3
gcd, e, y = extended_euclidean(d, (p-1)*(q-1))
print("Solution to gcd(a, b) = ax + by, with a=e, x=d=3 and b=(p-1)(q-1)")
print("gcd=", gcd, "e=", e, "y=", y)
print("e=", e, "is the modular inverse of d mod phi(n), and our public key component")
print("d*e - phi(n) = ", d*e-(p-1)*(q-1))

Solution to gcd(a, b) = ax + by, with a=e, x=d=3 and b=(p-1)(q-1)
gcd= 1 e= 868479 y= -1
e= 868479 is the modular inverse of d mod phi(n), and our public key component
d*e - phi(n) =  1


### c
Encrypt the message 111 using repeated squaring. Implement the algorithm yourself.
<hr>

N ≡ M<sup>e</sup> mod(n)

In [28]:
import math

In [43]:
def encrypt_rsa(m, e, n):
  return m

def find_exponent(e, pows):
  if(e == 1): return 0
  i = 1
  while e // math.pow(2, i) != 1:
    i+=1
  pows.append(i)
  find_exponent(e-math.pow(2, i), pows)


In [53]:
p = 1287
q = 2027
d = 3
e = 868479
n = p*q

pows = []

find_exponent(e, pows)

print(pows)
e_ = e
for pow in pows:
  print("e - 2**i", e_,"-", math.pow(2,pow), "=", int(e_ - math.pow(2, pow)))
  e_ -= math.pow(2, pow) 



[19, 18, 16, 14, 6, 5, 4, 3, 2, 1]
e - 2**i 868479 - 524288.0 = 344191
e - 2**i 344191.0 - 262144.0 = 82047
e - 2**i 82047.0 - 65536.0 = 16511
e - 2**i 16511.0 - 16384.0 = 127
e - 2**i 127.0 - 64.0 = 63
e - 2**i 63.0 - 32.0 = 31
e - 2**i 31.0 - 16.0 = 15
e - 2**i 15.0 - 8.0 = 7
e - 2**i 7.0 - 4.0 = 3
e - 2**i 3.0 - 2.0 = 1
